## Data loading

In [1]:
from utils import load_rumors_from_jsonl
import os

out_dir = './temp-data'

clef_path = '../clef2024-checkthat-lab/task5'
data_path = os.path.join(clef_path, 'data')

filepath_train = os.path.join(data_path, 'English_train.json')
filepath_dev = os.path.join(data_path, 'English_dev.json')

train_jsons = load_rumors_from_jsonl(filepath_train)
dev_jsons = load_rumors_from_jsonl(filepath_dev)

print(f'loaded {len(train_jsons)} training json objects and {len(dev_jsons)} dev objects')

loaded 96 training json objects and 32 dev objects


In [2]:
import json

print(json.dumps(train_jsons[1], indent=4))

{
    "id": "AuRED_037",
    "rumor": "Macron to Sky News: After my visit to Mrs. Fairouz last night and the visit to the Jaj Cedar Reserve, I realized the love of a large section of the Lebanese people for their President of the Republic. Fairouz also told me about her appreciation and love for the President and the reform project that the President of the Republic wants to implement. I also salute the President of the Republic and his efforts and patience. https://t.co/7pMab8yWCD",
    "label": "REFUTES",
    "timeline": [
        [
            "https://twitter.com/skynewsarabia",
            "1302029928867729411",
            "#Iraq.. Record infections with #Corona and hospitals are on the verge of collapse #Sky_News #Covid19 #covid19 https://t.co/zSr70lBncb"
        ],
        [
            "https://twitter.com/skynewsarabia",
            "1302028670446444544",
            "The drums of war are beating in the #Mediterranean. Follow us in the latest episodes of the #Sky_News_Room pr

In [3]:
import json

print(json.dumps(dev_jsons[0], indent=4))

{
    "id": "AuRED_142",
    "rumor": "Naturalization decree in preparation: Lebanese passports for sale?! https://t.co/UuQ7yMbSWJ https://t.co/Jf1K1NbZJD",
    "label": "REFUTES",
    "timeline": [
        [
            "https://twitter.com/LBpresidency",
            "1556600039211925504",
            "Today, the President of the Republic, General Michel Aoun, signed 9 laws that were previously approved by the House of Representatives. Details at the following link: https://t.co/wmrSaaEwDu"
        ],
        [
            "https://twitter.com/LBpresidency",
            "1556559119045332992",
            "President Aoun received the Minister of Foreign Affairs and Expatriates, Dr. Abdullah Bouhabib, and the Minister of Social Affairs, Hector Hajjar, and discussed with them developments related to the file of displaced Syrians in Lebanon https://t.co/QLQAJSKzs1"
        ],
        [
            "https://twitter.com/LBpresidency",
            "1556558220533157890",
            "Presiden

In [4]:
from clef.utils import combine_rumors_with_trec_file_judgements
import os

submission_path = os.path.join(clef_path, 'submission_samples', 'KGAT_zeroShot_evidence_English_dev.txt')
# submission_path = './temp-data/tfidf-trec.txt'

dev_jsons = combine_rumors_with_trec_file_judgements(dev_jsons, submission_path)

print(json.dumps(dev_jsons[2], indent=4))

{
    "id": "AuRED_132",
    "rumor": "*Riyad Salama to NBN, raising the value of the dollar through Resolution 158 from 8,000 LBP per dollar to 15,000 LBP, starting from the first of next month.* *Riyad Salama to NBN, we are raising the ceiling on banking withdrawals, starting from the first of next month, from $400 to $1,500 for individuals.*",
    "label": "REFUTES",
    "timeline": [
        [
            "https://twitter.com/nbntweets",
            "1591521645403029505",
            "Protests in #Germany demanding #Turkey to stop using chemical weapons in #Iraqi Kurdistan https://t.co/URM6stQ2sj https://t.co/8uxLDgH5yD"
        ],
        [
            "https://twitter.com/nbntweets",
            "1591519539355082752",
            "Urgent - The Pakistani Foreign Ministry announces the postponement of the visit of the Saudi Crown Prince to Islamabad"
        ],
        [
            "https://twitter.com/nbntweets",
            "1591517950657581057",
            "Reopening #Beddawi 

## NLI

In [5]:
from transformers import pipeline
from typing import Dict, NamedTuple, TypedDict, Union

class VerificationResult(NamedTuple):
    label: str
    score: float

# Initialize the NLI pipeline with a pre-trained model
# nli_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
nli_pipeline = pipeline("text-classification", model="roberta-large-mnli")

def check_statement_with_evidence(statement: str, evidence: str) -> VerificationResult:
    # Define the candidate labels for NLI
    # candidate_labels = ["SUPPORTS", "REFUTES"]
    input_text = f"{evidence} [SEP] {statement}"

    # Use the NLI pipeline to predict the relationship
    # result = nli_pipeline(evidence, hypothesis=statement, candidate_labels=candidate_labels, multi_label=False)
    result = nli_pipeline(input_text)

    label_map = {
        "CONTRADICTION": "REFUTES",
        "NEUTRAL": "NOT ENOUGH INFO",
        "ENTAILMENT": "SUPPORTS"
    }

    label = label_map[result[0]['label']]
    score = result[0]['score']
    # Return the result
    return (label, score)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from typing import List

from clef.utils import RankedDocs

def factcheck_using_evidence(claim: str, evidence: List[RankedDocs]):
    """
    Predict a judgement for a rumor using the retrieved evidence.

    Parameters:
        - rumor_dict (Dict): a Python Dict of a single rumor from the dataset, extended by the key 'rerieved_evidence'
    """
    predicted_evidence = []
    confidences = []

    for author_account, tweet_id, evidence_text, rank, score in evidence:
        label, confidence = check_statement_with_evidence(claim, evidence_text)

        # CLEF CheckThat! task 5: score is [-1, +1] where 
        #   -1 means evidence strongly refuted
        #   +1 means evidence strongly supports

        if label == "REFUTES":
            confidence *= -1
        elif label == "NOT ENOUGH INFO":
            confidence = 0 # TODO uhmmm...

        predicted_evidence += [[
            author_account,
            tweet_id,
            evidence_text,
            confidence,
        ]]

        confidences += [confidence]

    meanconf = sum(confidences) / len(confidences) # mean confidence, no weighting
    
    if meanconf > 0.3:
        pred_label = "SUPPORTS"
    elif meanconf < -0.3:
        pred_label = "REFUTES"
    else:
        pred_label = "NOT ENOUGH INFO"

    return pred_label, predicted_evidence

In [7]:
# individual step

from utils import write_jsonlines_from_dicts
from tqdm import tqdm

res_jsons = []

for item in tqdm(dev_jsons[:]):
    rumor = item["rumor"]
    retrieved_evidence = item["retrieved_evidence"]

    if retrieved_evidence: # only run fact check if we actually have retrieved evidence
        pred_label, pred_evidence = factcheck_using_evidence(rumor, retrieved_evidence)

        res_jsons += [
            {
                "id": item["id"],
                "label": item["label"],
                "claim": item["rumor"],
                "predicted_label": pred_label,
                "predicted_evidence": pred_evidence,
            }
        ]

outfile = "temp-data/zeroshot-ver.jsonl"
write_jsonlines_from_dicts(outfile, res_jsons)

  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [01:31<00:00,  2.85s/it]


## Evaluation

In [8]:
from scoring_utils import eval_run

task5_dir = '../clef2024-checkthat-lab/task5'

sample_submission_file = task5_dir + '/submission_samples/KGAT_zeroShot_verification_English_dev.json'

nli_submission_file = 'temp-data/zeroshot-ver.jsonl'
ground_truth_file = task5_dir + '/data/Arabic_dev.json'
out_file = 'temp-data/out.csv'

print('sample')
eval_run(sample_submission_file,ground_truth_file, out_file)

print('nli')
eval_run(nli_submission_file,ground_truth_file, out_file)

sample
Macro_F1 0.5081585081585082
Strict Macro_F1 0.5081585081585082
nli
Macro_F1 0.23049095607235145
Strict Macro_F1 0.18604651162790697
